# Import Modules and Libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib
import numpy as np
import imageio
from matplotlib import pyplot as plt
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import make_grid, save_image

# MNIST Data

In [2]:
bs = 100
# Import MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=False)

# Data Loader 
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

# VAE Class

In [3]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        
        # encoder part
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc31 = nn.Linear(h_dim2, z_dim)
        self.fc32 = nn.Linear(h_dim2, z_dim)
        # decoder part
        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)
        
    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc31(h), self.fc32(h) # mu, log_var
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
        
    def decoder(self, z):
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return F.sigmoid(self.fc6(h)) 
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

# build model
vae = VAE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=2)
if torch.cuda.is_available():
    vae.cuda()

# Optimizer

In [4]:
optimizer = optim.Adam(vae.parameters())
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

# Training

In [5]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.cuda()
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

# Evaluation

In [6]:
def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.cuda()
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))
    return(test_loss)

In [13]:
test_loss_array = []
images = []
z = torch.randn(64, 2).cuda()
for epoch in range(1, 301):
    train(epoch)
    test_loss = test()
    test_loss_array.append(test_loss)
    with torch.no_grad():
        sample = vae.decoder(z).cpu().detach()
        save_image(sample.view(64, 1, 28, 28), f"./outputs/gen_img_{epoch}.png")
        images.append(make_grid(sample.view(64, 1, 28, 28)))
to_pil_image = transforms.ToPILImage()
imgs = [np.array(to_pil_image(img)) for img in images]
imageio.mimsave('./outputs/generator_images.gif', imgs)

Train Epoch: 1 [0/60000 (0%)]	Loss: 139.184150
Train Epoch: 1 [10000/60000 (17%)]	Loss: 130.665088
Train Epoch: 1 [20000/60000 (33%)]	Loss: 139.688008
Train Epoch: 1 [30000/60000 (50%)]	Loss: 144.890029
Train Epoch: 1 [40000/60000 (67%)]	Loss: 137.411348
Train Epoch: 1 [50000/60000 (83%)]	Loss: 145.081807
====> Epoch: 1 Average loss: 144.1354
====> Test set loss: 144.7856
Train Epoch: 2 [0/60000 (0%)]	Loss: 142.658301
Train Epoch: 2 [10000/60000 (17%)]	Loss: 137.097891
Train Epoch: 2 [20000/60000 (33%)]	Loss: 147.373848
Train Epoch: 2 [30000/60000 (50%)]	Loss: 149.713564
Train Epoch: 2 [40000/60000 (67%)]	Loss: 142.482402
Train Epoch: 2 [50000/60000 (83%)]	Loss: 141.689238
====> Epoch: 2 Average loss: 143.4704
====> Test set loss: 144.3308
Train Epoch: 3 [0/60000 (0%)]	Loss: 142.327559
Train Epoch: 3 [10000/60000 (17%)]	Loss: 138.916025
Train Epoch: 3 [20000/60000 (33%)]	Loss: 136.812383
Train Epoch: 3 [30000/60000 (50%)]	Loss: 141.625742
Train Epoch: 3 [40000/60000 (67%)]	Loss: 146.74

Train Epoch: 22 [50000/60000 (83%)]	Loss: 138.873467
====> Epoch: 22 Average loss: 137.1684
====> Test set loss: 139.3342
Train Epoch: 23 [0/60000 (0%)]	Loss: 135.760625
Train Epoch: 23 [10000/60000 (17%)]	Loss: 133.044609
Train Epoch: 23 [20000/60000 (33%)]	Loss: 141.970488
Train Epoch: 23 [30000/60000 (50%)]	Loss: 130.458027
Train Epoch: 23 [40000/60000 (67%)]	Loss: 130.343486
Train Epoch: 23 [50000/60000 (83%)]	Loss: 133.177656
====> Epoch: 23 Average loss: 137.1659
====> Test set loss: 139.5767
Train Epoch: 24 [0/60000 (0%)]	Loss: 132.526211
Train Epoch: 24 [10000/60000 (17%)]	Loss: 129.637705
Train Epoch: 24 [20000/60000 (33%)]	Loss: 138.051318
Train Epoch: 24 [30000/60000 (50%)]	Loss: 141.070010
Train Epoch: 24 [40000/60000 (67%)]	Loss: 138.244414
Train Epoch: 24 [50000/60000 (83%)]	Loss: 141.962949
====> Epoch: 24 Average loss: 137.0491
====> Test set loss: 139.7014
Train Epoch: 25 [0/60000 (0%)]	Loss: 144.188877
Train Epoch: 25 [10000/60000 (17%)]	Loss: 141.678887
Train Epoch: 

Train Epoch: 44 [20000/60000 (33%)]	Loss: 140.290254
Train Epoch: 44 [30000/60000 (50%)]	Loss: 136.581406
Train Epoch: 44 [40000/60000 (67%)]	Loss: 141.188477
Train Epoch: 44 [50000/60000 (83%)]	Loss: 135.267109
====> Epoch: 44 Average loss: 135.0337
====> Test set loss: 137.9329
Train Epoch: 45 [0/60000 (0%)]	Loss: 133.809551
Train Epoch: 45 [10000/60000 (17%)]	Loss: 132.866152
Train Epoch: 45 [20000/60000 (33%)]	Loss: 130.211211
Train Epoch: 45 [30000/60000 (50%)]	Loss: 140.994082
Train Epoch: 45 [40000/60000 (67%)]	Loss: 130.259648
Train Epoch: 45 [50000/60000 (83%)]	Loss: 132.433184
====> Epoch: 45 Average loss: 135.2689
====> Test set loss: 138.7123
Train Epoch: 46 [0/60000 (0%)]	Loss: 137.474580
Train Epoch: 46 [10000/60000 (17%)]	Loss: 134.379287
Train Epoch: 46 [20000/60000 (33%)]	Loss: 134.455391
Train Epoch: 46 [30000/60000 (50%)]	Loss: 128.017725
Train Epoch: 46 [40000/60000 (67%)]	Loss: 131.530869
Train Epoch: 46 [50000/60000 (83%)]	Loss: 141.283633
====> Epoch: 46 Average 

====> Epoch: 65 Average loss: 134.3928
====> Test set loss: 137.4362
Train Epoch: 66 [0/60000 (0%)]	Loss: 131.963037
Train Epoch: 66 [10000/60000 (17%)]	Loss: 137.706650
Train Epoch: 66 [20000/60000 (33%)]	Loss: 134.419932
Train Epoch: 66 [30000/60000 (50%)]	Loss: 140.229355
Train Epoch: 66 [40000/60000 (67%)]	Loss: 132.302676
Train Epoch: 66 [50000/60000 (83%)]	Loss: 134.096387
====> Epoch: 66 Average loss: 133.5712
====> Test set loss: 138.0400
Train Epoch: 67 [0/60000 (0%)]	Loss: 133.976387
Train Epoch: 67 [10000/60000 (17%)]	Loss: 135.579111
Train Epoch: 67 [20000/60000 (33%)]	Loss: 139.153564
Train Epoch: 67 [30000/60000 (50%)]	Loss: 135.084287
Train Epoch: 67 [40000/60000 (67%)]	Loss: 135.537686
Train Epoch: 67 [50000/60000 (83%)]	Loss: 127.993164
====> Epoch: 67 Average loss: 133.4467
====> Test set loss: 137.8300
Train Epoch: 68 [0/60000 (0%)]	Loss: 135.830967
Train Epoch: 68 [10000/60000 (17%)]	Loss: 138.543633
Train Epoch: 68 [20000/60000 (33%)]	Loss: 127.741563
Train Epoch: 

Train Epoch: 87 [30000/60000 (50%)]	Loss: 131.877354
Train Epoch: 87 [40000/60000 (67%)]	Loss: 131.564453
Train Epoch: 87 [50000/60000 (83%)]	Loss: 125.826553
====> Epoch: 87 Average loss: 132.7432
====> Test set loss: 138.3582
Train Epoch: 88 [0/60000 (0%)]	Loss: 129.302217
Train Epoch: 88 [10000/60000 (17%)]	Loss: 130.290400
Train Epoch: 88 [20000/60000 (33%)]	Loss: 137.897383
Train Epoch: 88 [30000/60000 (50%)]	Loss: 136.129580
Train Epoch: 88 [40000/60000 (67%)]	Loss: 128.280391
Train Epoch: 88 [50000/60000 (83%)]	Loss: 139.976846
====> Epoch: 88 Average loss: 132.4969
====> Test set loss: 137.6677
Train Epoch: 89 [0/60000 (0%)]	Loss: 124.284111
Train Epoch: 89 [10000/60000 (17%)]	Loss: 128.422646
Train Epoch: 89 [20000/60000 (33%)]	Loss: 135.651201
Train Epoch: 89 [30000/60000 (50%)]	Loss: 135.897832
Train Epoch: 89 [40000/60000 (67%)]	Loss: 136.695527
Train Epoch: 89 [50000/60000 (83%)]	Loss: 130.987070
====> Epoch: 89 Average loss: 132.5842
====> Test set loss: 138.1423
Train Ep

...

Train Epoch: 214 [40000/60000 (67%)]	Loss: 125.156191
Train Epoch: 214 [50000/60000 (83%)]	Loss: 132.295098
====> Epoch: 214 Average loss: 130.0714
====> Test set loss: 138.5886
Train Epoch: 215 [0/60000 (0%)]	Loss: 128.384463
Train Epoch: 215 [10000/60000 (17%)]	Loss: 124.319414
Train Epoch: 215 [20000/60000 (33%)]	Loss: 127.883301
Train Epoch: 215 [30000/60000 (50%)]	Loss: 132.238398
Train Epoch: 215 [40000/60000 (67%)]	Loss: 130.630596
Train Epoch: 215 [50000/60000 (83%)]	Loss: 122.882246
====> Epoch: 215 Average loss: 129.9415
====> Test set loss: 138.7458
Train Epoch: 216 [0/60000 (0%)]	Loss: 131.077871
Train Epoch: 216 [10000/60000 (17%)]	Loss: 127.879004
Train Epoch: 216 [20000/60000 (33%)]	Loss: 120.319941
Train Epoch: 216 [30000/60000 (50%)]	Loss: 125.603027
Train Epoch: 216 [40000/60000 (67%)]	Loss: 131.456660
Train Epoch: 216 [50000/60000 (83%)]	Loss: 129.722158
====> Epoch: 216 Average loss: 129.9529
====> Test set loss: 138.7886
Train Epoch: 217 [0/60000 (0%)]	Loss: 125.90

Train Epoch: 235 [50000/60000 (83%)]	Loss: 134.215391
====> Epoch: 235 Average loss: 130.1590
====> Test set loss: 138.7983
Train Epoch: 236 [0/60000 (0%)]	Loss: 134.532617
Train Epoch: 236 [10000/60000 (17%)]	Loss: 120.335361
Train Epoch: 236 [20000/60000 (33%)]	Loss: 122.341689
Train Epoch: 236 [30000/60000 (50%)]	Loss: 134.427021
Train Epoch: 236 [40000/60000 (67%)]	Loss: 132.721494
Train Epoch: 236 [50000/60000 (83%)]	Loss: 124.484629
====> Epoch: 236 Average loss: 130.0242
====> Test set loss: 139.1281
Train Epoch: 237 [0/60000 (0%)]	Loss: 127.944639
Train Epoch: 237 [10000/60000 (17%)]	Loss: 126.675215
Train Epoch: 237 [20000/60000 (33%)]	Loss: 136.198887
Train Epoch: 237 [30000/60000 (50%)]	Loss: 124.025293
Train Epoch: 237 [40000/60000 (67%)]	Loss: 130.544766
Train Epoch: 237 [50000/60000 (83%)]	Loss: 139.218320
====> Epoch: 237 Average loss: 130.0689
====> Test set loss: 138.6701
Train Epoch: 238 [0/60000 (0%)]	Loss: 131.003896
Train Epoch: 238 [10000/60000 (17%)]	Loss: 128.55

====> Epoch: 256 Average loss: 129.3535
====> Test set loss: 139.3266
Train Epoch: 257 [0/60000 (0%)]	Loss: 134.972148
Train Epoch: 257 [10000/60000 (17%)]	Loss: 128.535557
Train Epoch: 257 [20000/60000 (33%)]	Loss: 131.708984
Train Epoch: 257 [30000/60000 (50%)]	Loss: 121.322598
Train Epoch: 257 [40000/60000 (67%)]	Loss: 128.548877
Train Epoch: 257 [50000/60000 (83%)]	Loss: 121.617393
====> Epoch: 257 Average loss: 129.4988
====> Test set loss: 139.1232
Train Epoch: 258 [0/60000 (0%)]	Loss: 129.699375
Train Epoch: 258 [10000/60000 (17%)]	Loss: 123.423799
Train Epoch: 258 [20000/60000 (33%)]	Loss: 125.024443
Train Epoch: 258 [30000/60000 (50%)]	Loss: 131.000420
Train Epoch: 258 [40000/60000 (67%)]	Loss: 129.538301
Train Epoch: 258 [50000/60000 (83%)]	Loss: 126.726797
====> Epoch: 258 Average loss: 129.9021
====> Test set loss: 139.1560
Train Epoch: 259 [0/60000 (0%)]	Loss: 134.390479
Train Epoch: 259 [10000/60000 (17%)]	Loss: 123.674365
Train Epoch: 259 [20000/60000 (33%)]	Loss: 135.54

====> Test set loss: 139.9408
Train Epoch: 278 [0/60000 (0%)]	Loss: 130.065977
Train Epoch: 278 [10000/60000 (17%)]	Loss: 128.327617
Train Epoch: 278 [20000/60000 (33%)]	Loss: 125.993223
Train Epoch: 278 [30000/60000 (50%)]	Loss: 139.160283
Train Epoch: 278 [40000/60000 (67%)]	Loss: 132.372637
Train Epoch: 278 [50000/60000 (83%)]	Loss: 130.408330
====> Epoch: 278 Average loss: 130.3080
====> Test set loss: 139.1269
Train Epoch: 279 [0/60000 (0%)]	Loss: 133.199248
Train Epoch: 279 [10000/60000 (17%)]	Loss: 134.018076
Train Epoch: 279 [20000/60000 (33%)]	Loss: 132.153662
Train Epoch: 279 [30000/60000 (50%)]	Loss: 132.861826
Train Epoch: 279 [40000/60000 (67%)]	Loss: 132.151064
Train Epoch: 279 [50000/60000 (83%)]	Loss: 126.385332
====> Epoch: 279 Average loss: 129.6780
====> Test set loss: 139.2284
Train Epoch: 280 [0/60000 (0%)]	Loss: 134.004414
Train Epoch: 280 [10000/60000 (17%)]	Loss: 132.118223
Train Epoch: 280 [20000/60000 (33%)]	Loss: 133.545479
Train Epoch: 280 [30000/60000 (50%)

Train Epoch: 299 [10000/60000 (17%)]	Loss: 129.739824
Train Epoch: 299 [20000/60000 (33%)]	Loss: 136.022100
Train Epoch: 299 [30000/60000 (50%)]	Loss: 130.411328
Train Epoch: 299 [40000/60000 (67%)]	Loss: 136.679219
Train Epoch: 299 [50000/60000 (83%)]	Loss: 127.274053
====> Epoch: 299 Average loss: 129.2232
====> Test set loss: 140.0319
Train Epoch: 300 [0/60000 (0%)]	Loss: 131.347383
Train Epoch: 300 [10000/60000 (17%)]	Loss: 127.702217
Train Epoch: 300 [20000/60000 (33%)]	Loss: 130.046953
Train Epoch: 300 [30000/60000 (50%)]	Loss: 138.316758
Train Epoch: 300 [40000/60000 (67%)]	Loss: 126.883066
Train Epoch: 300 [50000/60000 (83%)]	Loss: 135.658779
====> Epoch: 300 Average loss: 129.3788
====> Test set loss: 139.8180


In [ ]:
plt.figure()
plt.plot(test_loss_array, label='Reconstruction loss')
plt.legend()

# Generate

In [ ]:
with torch.no_grad():
    z = torch.randn(64, 2).cuda()
    sample = vae.decoder(z).cuda()
    
    save_image(sample.view(64, 1, 28, 28), './outputs/gen_img_{}.png')